### This is a new version of MLEAP scripts, started in late Aug 2022.
It will combine IProject_MLEAP_ANN and IP_MLEAP script, while improving them.

#### Outline

1. Load libraries and data.
2. pEDA. Look at feature distribution, fix them if they do not look right.
3. Train-test split. Most likely couple years into test set. 2015-2018?. Impute missing values.
4. Transform numerical features, add ohe for inds.
5. Fit classic models: ols as a baseline, then xgb.
6. Fir DL.


Notes:
ideally, I want to use time-based cross-validation.
since I have panel data, it is not a trivial task.
need to find some solution online.
e.g., https://towardsdatascience.com/time-based-cross-validation-d259b13d42b8.

for now, will try to do siple for loop.


In [27]:
# 0. Import libraries #

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time, math, re, warnings, random, gc, dill, optuna, pickle
import statsmodels.api as sm
from random import sample

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance
from category_encoders import MEstimateEncoder
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use('seaborn-white')
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', 110)
gc.enable()

In [28]:
### target encoding ###
# source: https://www.kaggle.com/code/ryanholbrook/feature-engineering-for-house-prices/notebook

class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=4)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [29]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [30]:
# for loop to see appx performance over the whole sample with some rolling window

time0 = time.time()

min_prd_list = range(100, 676, 25)
windows_width = 3*12
cv_regularizer=0.04
optuna_trials = 20

results = pd.DataFrame(columns = ['min_prd', 'xgbf', 'xgbgs', 'xgbo'])
results.min_prd = min_prd_list

for min_prd in min_prd_list:
    
    
    with open('../input/kaggle-46pkl/IMLEAP_v4.pkl', 'rb') as pickled_one:
        df = pickle.load(pickled_one)
    df = df[df.prd.isin(range(min_prd-1, min_prd+windows_width+2))]
    df_cnt = df.count()
    empty_cols = list(df_cnt[df_cnt<int(df.shape[0]/2)].index)
    df.drop(columns=empty_cols, inplace=True)
    display(df.shape, df.head(), df.year.describe(), df.count())
    
    df = df[(df.RET>-50)&(df.RET<75)]
    meanret = df.groupby('prd').RET.mean().to_frame().reset_index().rename(columns={'RET':'mRET'})
    df = pd.merge(df, meanret, on='prd', how='left')
    df.RET = df.RET-df.mRET
    df.drop(columns='mRET', inplace=True)

    features_miss_dummies = ['amhd', 'BAspr']
    for col in features_miss_dummies:
        if col in df.columns:
            df[col+'_miss'] = df[col].isnull().astype(int)

    temp_cols = ['PERMNO', 'prd', 'year']
    train = df[df.prd<(min_prd+windows_width)]
    test = df[df.prd==(min_prd+windows_width)]
    train.drop(columns=temp_cols, inplace=True)
    test.drop(columns=temp_cols, inplace=True)

    col_ignore = ['RET']
    col_cat = ['ind']
    col_num = [x for x in train.columns if x not in col_ignore+col_cat]
    for col in col_num:
        train[col] = train[col].fillna(train[col].median())
        test[col] = test[col].fillna(train[col].median())
    for col in col_cat:
        train[col] = train[col].fillna(value=-1000)
        test[col] = test[col].fillna(value=-1000)

    X_train = train.copy()
    y_train = X_train.pop('RET')
    X_test = test.copy()
    y_test = X_test.pop('RET')

    feature_transformer = ColumnTransformer([('num', StandardScaler(), col_num),
                                            ("cat", OneHotEncoder(sparse = False, handle_unknown="ignore", drop='if_binary'), col_cat)], 
                                            remainder="passthrough")

    print('Number of features before transformation: ', X_train.shape)
    X_train = pd.DataFrame(feature_transformer.fit_transform(X_train), columns=feature_transformer.get_feature_names_out())
    X_test = pd.DataFrame(feature_transformer.transform(X_test), columns=feature_transformer.get_feature_names_out())
    print('time to do feature proprocessing: ')
    print('Number of features after transformation: ', X_train.shape)

    print('mae of a constant model', mean_absolute_error(df.RET, np.ones(df.shape[0])*(df.RET.mean())))
    print('R2 of a constant model', r2_score(df.RET, np.ones(df.shape[0])*(df.RET.mean())))

    xgb1 = XGBRegressor(tree_method = 'gpu_hist', n_estimators=300, max_depth=5, eta=0.03, colsample_bytree=0.6)
    xgb1.fit(X_train, y_train)
    print('XGB train:', mean_absolute_error(y_train, xgb1.predict(X_train)), r2_score(y_train, xgb1.predict(X_train)))

    time1 = time.time()
    xgb = XGBRegressor(tree_method = 'gpu_hist')
    param_grid = {'n_estimators':[400, 700], 'max_depth':[2,3,4], 'eta':[0.006, 0.012, 0.02], 'subsample':[0.6], 'colsample_bytree':[0.6]}
    xgbm = GridSearchCV(xgb, param_grid, cv=2, verbose=2, scoring='r2')
    xgbm.fit(X_train, y_train)
    print('XGB', xgbm.best_params_, xgbm.best_score_, time.time()-time1)
    print('XGB train:', mean_absolute_error(y_train, xgbm.predict(X_train)), r2_score(y_train, xgbm.predict(X_train)), time.time()-time1)

    time1 = time.time()
    def objective(trial, cv_runs=1, n_splits=2, n_jobs=-1):

        params = {
        "tree_method": 'gpu_hist',
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "n_estimators": trial.suggest_int("n_estimators", 500, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_uniform("learning_rate", 0.001, 0.05),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 0.95),
        "subsample": trial.suggest_uniform("subsample", 0.3, 0.95),
        "alpha": trial.suggest_loguniform("alpha", 0.1, 30.0),
        "lambda": trial.suggest_loguniform("lambda", 0.1, 200.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-10, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 50)    }

        temp_out = []

        for i in range(cv_runs):

            X = X_train
            y = y_train
            model = XGBRegressor(**params, njobs=-1)
            rkf = KFold(n_splits=n_splits, shuffle=True)
            X_values = X.values
            y_values = y.values
            y_pred = np.zeros_like(y_values)
            y_pred_train = np.zeros_like(y_values)
            for train_index, test_index in rkf.split(X_values):
                X_A, X_B = X_values[train_index, :], X_values[test_index, :]
                y_A, y_B = y_values[train_index], y_values[test_index]
                model.fit(X_A, y_A, eval_set=[(X_B, y_B)], verbose = False)
                y_pred[test_index] = model.predict(X_B)
                y_pred_train[train_index] = model.predict(X_A)

            score_train = r2_score(y_train, y_pred_train)
            score_test = r2_score(y_train, y_pred) 
            overfit = (score_train-score_test)
            temp_out.append(score_test-cv_regularizer*overfit)

        return (np.mean(temp_out))

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=optuna_trials)
    print('Total time for hypermarameter optimization ', time.time()-time1)
    hp = study.best_params
    for key, value in hp.items():
        print(f"{key:>20s} : {value}")
    print(f"{'best objective value':>20s} : {study.best_value}")
    optuna_hyperpars = study.best_params
    optuna_hyperpars['tree_method']='gpu_hist'
    optuna_xgb = XGBRegressor(**optuna_hyperpars)
    optuna_xgb.fit(X_train, y_train)
    print('Optuna XGB train:', 
          mean_absolute_error(y_train, optuna_xgb.predict(X_train)), r2_score(y_train, optuna_xgb.predict(X_train)), time.time()-time1)

    # Evaluate performance of XGB models:
    r2_xgb1 = r2_score(y_test, xgb1.predict(X_test))
    r2_xgbgs = r2_score(y_test, xgbm.predict(X_test))
    r2_xgbo = r2_score(y_test, optuna_xgb.predict(X_test))

    print('Min_prd: ', min_prd)
    print('Constant guess: ', mean_absolute_error(y_test, np.ones(len(y_test))*y_test.mean()), 
          r2_score(y_test, np.ones(len(y_test))*y_test.mean()))
    print('XGB test:', mean_absolute_error(y_test, xgb1.predict(X_test)), r2_xgb1)
    print('XGB GS test:', mean_absolute_error(y_test, xgbm.predict(X_test)), r2_xgbgs)
    print('Optuna XGB test:', mean_absolute_error(y_test, optuna_xgb.predict(X_test)), r2_xgbo)

    results.loc[results.min_prd==min_prd,'xgbf':'xgbo'] = r2_xgb1, r2_xgbgs, r2_xgbo
    
print(time.time()-time0, results)

(31363, 40)

,PERMNO,prd,mom242,year,RET,ind,bm,op,gp,inv,mom11,mom122,amhd,ivol_capm,ivol_ff5,beta_bw,MAX,vol1m,vol6m,vol12m,size,lbm,lop,lgp,linv,llme,l1amhd,l1MAX,l3amhd,l3MAX,l6amhd,l6MAX,l12amhd,l12MAX,l12mom122,l12ivol_capm,l12ivol_ff5,l12beta_bw,l12vol6m,l12vol12m
24,10006,99,30.007610,1966,-1.8425,25.0,-0.483038,0.179707,0.260061,0.077403,5.4095,23.511075,1.475168,1.402148,1.282759,1.336867,3.3976,1.506242,1.833065,1.648786,5.695347,-0.417907,0.135547,0.222194,0.067345,5.434935,1.516443,2.6538,1.479387,4.2841,1.436185,4.0548,1.669988,2.6538,18.881331,0.765013,0.702024,1.195248,1.193920,1.231535
25,10006,100,22.973353,1966,-13.9454,25.0,-0.483038,0.179707,0.260061,0.077403,-1.8425,29.337614,1.478980,0.790153,0.754158,1.239788,1.4671,0.877792,1.758302,1.629121,5.680309,-0.417907,0.135547,0.222194,0.067345,5.444597,1.475168,3.3976,1.498098,3.1745,1.411486,4.5856,1.606326,3.3976,-0.948286,0.890444,0.810150,1.250279,1.236353,1.235768
26,10006,101,0.562306,1966,-4.8379,25.0,-0.483038,0.179707,0.260061,0.077403,-13.9454,20.186279,1.582242,1.380279,1.368466,1.238541,2.1292,2.117274,1.705033,1.723923,5.521941,-0.417907,0.135547,0.222194,0.067345,5.491554,1.478980,1.4671,1.516443,2.6538,1.439482,3.1527,1.516363,1.4671,-1.278890,0.985169,0.908360,1.250547,1.195605,1.222002
27,10006,102,-6.780997,1966,-2.9268,25.0,-0.483038,0.179707,0.260061,0.077403,-4.8379,5.036237,1.641163,1.941973,1.031749,1.220998,5.3222,2.173122,1.674638,1.793334,5.476547,-0.417907,0.135547,0.222194,0.067345,5.479246,1.582242,2.1292,1.475168,3.3976,1.479387,4.2841,1.522883,2.1292,-2.716252,1.199252,1.070752,1.265243,1.204190,1.233468
28,10006,103,-8.513334,1966,0.9968,25.0,-0.483038,0.179707,0.260061,0.077403,-2.9268,-9.564465,1.713909,2.434331,1.877713,1.166406,7.8011,2.697039,1.892122,1.886296,5.451468,-0.417907,0.135547,0.222194,0.067345,5.582127,1.641163,5.3222,1.478980,1.4671,1.498098,3.1745,1.485769,5.3222,-6.697673,1.477127,1.395748,1.287226,1.312899,1.275484


count    31363.000000
mean      1967.710965
std          1.003547
min       1966.000000
25%       1967.000000
50%       1968.000000
75%       1969.000000
max       1969.000000
Name: year, dtype: float64

PERMNO          31363
prd             31363
mom242          31136
year            31363
RET             31363
ind             31363
bm              31363
op              31363
gp              31363
inv             31363
mom11           31363
mom122          31363
amhd            29013
ivol_capm       31361
ivol_ff5        31361
beta_bw         31363
MAX             31363
vol1m           31360
vol6m           31357
vol12m          31350
size            31363
lbm             31363
lop             31363
lgp             31363
linv            31363
llme            31363
l1amhd          28926
l1MAX           31362
l3amhd          28737
l3MAX           31360
l6amhd          28420
l6MAX           31357
l12amhd         27713
l12MAX          31362
l12mom122       31191
l12ivol_capm    31349
l12ivol_ff5     31349
l12beta_bw      31352
l12vol6m        31339
l12vol12m       31166
dtype: int64

Number of features before transformation:  (29248, 37)
time to do feature proprocessing: 
Number of features after transformation:  (29248, 81)
mae of a constant model 6.553607277016165
R2 of a constant model 0.0
XGB train: 6.110803010839697 0.1613195732684598
Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=400, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=400, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=700, subsample=0.6; total time=   0.8s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=700, subsample=0.6; total time=   0.8s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=3, n_estimators=400, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=3, n_estimators=400, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=0.6, eta=0

...

Number of features before transformation:  (57612, 44)
time to do feature proprocessing: 
Number of features after transformation:  (57612, 92)
mae of a constant model 7.844483176058157
R2 of a constant model 0.0
XGB train: 7.463082080313984 0.12777261074441615
Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=400, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=400, subsample=0.6; total time=   0.6s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=700, subsample=0.6; total time=   1.0s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=2, n_estimators=700, subsample=0.6; total time=   0.9s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=3, n_estimators=400, subsample=0.6; total time=   0.8s
[CV] END colsample_bytree=0.6, eta=0.006, max_depth=3, n_estimators=400, subsample=0.6; total time=   0.8s
[CV] END colsample_bytree=0.6, eta=

[I 2022-08-25 21:25:49,601] A new study created in memory with name: no-name-03400079-7a3f-44ca-8908-de03a1096848


XGB train: 7.698325588684248 0.05027393848590822 41.73289680480957


[I 2022-08-25 21:25:58,855] Trial 0 finished with value: -0.017507152426829303 and parameters: {'n_estimators': 944, 'max_depth': 5, 'learning_rate': 0.02173842376668825, 'colsample_bytree': 0.7059334365524941, 'subsample': 0.4516252398816166, 'alpha': 19.95713097370607, 'lambda': 0.4772871198201621, 'gamma': 9.308233133556265e-05, 'min_child_weight': 6.005904061507015}. Best is trial 0 with value: -0.017507152426829303.
[I 2022-08-25 21:26:05,029] Trial 1 finished with value: 0.004789122681059692 and parameters: {'n_estimators': 932, 'max_depth': 4, 'learning_rate': 0.019002484298095177, 'colsample_bytree': 0.5365596807266103, 'subsample': 0.6844164371402591, 'alpha': 23.59544126090816, 'lambda': 38.07860697981912, 'gamma': 1.6254762635028171e-07, 'min_child_weight': 4.237638540528578}. Best is trial 1 with value: 0.004789122681059692.
[I 2022-08-25 21:26:11,215] Trial 2 finished with value: 0.0010626717378536907 and parameters: {'n_estimators': 652, 'max_depth': 5, 'learning_rate': 0

Total time for hypermarameter optimization  92.27102971076965
        n_estimators : 601
           max_depth : 3
       learning_rate : 0.015196251991589521
    colsample_bytree : 0.41612524983927524
           subsample : 0.764846491142368
               alpha : 0.5476846156741678
              lambda : 25.534537192933342
               gamma : 2.2101698854530036e-09
    min_child_weight : 1.540181610474194
best objective value : 0.011918020633176344
Optuna XGB train: 7.7165973749988765 0.04007504337618861 93.96981620788574
Min_prd:  675
Constant guess:  6.559515936576189 0.0
XGB test: 6.606500270058161 -0.025128448860597175
XGB GS test: 6.587640379139121 -0.010631009835597505
Optuna XGB test: 6.5926129951965695 -0.008785808551218466
3660.1989362239838     min_prd      xgbf     xgbgs      xgbo
0       100 -0.033926 -0.019374 -0.024648
1       125  0.001887  0.006944  0.004293
2       150  0.009361  0.018258  0.019385
3       175 -0.008671 -0.002783  -0.00082
4       200  0.012099   0

In [33]:
results

,min_prd,xgbf,xgbgs,xgbo
0,100,-0.033926,-0.019374,-0.024648
1,125,0.001887,0.006944,0.004293
2,150,0.009361,0.018258,0.019385
3,175,-0.008671,-0.002783,-0.00082
4,200,0.012099,0.01089,0.007637
5,225,0.024659,0.024404,0.024533
6,250,-0.004759,0.003918,0.005939
7,275,0.013317,0.017266,0.022634
8,300,-0.009404,-0.000589,-0.001855
9,325,-0.00607,0.004755,0.006592


In [31]:
print('Total time for a script: ', time.time()-time0)

Total time for a script:  3660.214448451996


In [34]:
results.iloc[:,1:].mean()

xgbf     0.004061
xgbgs    0.008791
xgbo     0.008833
dtype: float64

In [ ]:
# 3yr window, trials=20, cv_reg=0.03: 0.88%. runs 1 hr.
